## 3rd Project | Machine Learning

###  Instructions

* Follow a Python Project structure.
* Work with a training and validation dataset to optimize & test your trading strategies using the datasets provided in the introduction section.
* The **ML models** that we'll be using are **Logistic Regression, Support Vector Machine & XGBoost**.
* Define the **independent** and **dependent** variables to train the models, remember that you can add any technical indicator to your dataset.
* Split the `train` datasets into train/test.
* Our dependent variable should be a category that we want to predict, i.e. "Buy" and "Not buy", or "Sell" and "Not sell" for the short models, we can construct it if the next k price is over / under a certain threshold.
* For each model, fine tune all hyperparameters worth moving, then you can easily generate the True / False signals to backtest.
* Be careful when selecting a metric to fine-tune.
* For each dataset train/test pair (1d, 1h, 5m, 1m):

    * Use the buy/sell signals from the predictions.
    * Create all possible combinations of all machine learning models (2^n - 1, 7...).
    * Backtest the strategies while keeping track of the operations and cash/portfolio value time series, remember that we'll be opening long & short positions.
    * Optimize the backtest parameters (TPE, Grid Search, PSO, Genetic Algorithms, ...), stop-loss/take-profit, volume of the trade, maximizing the profit of the strategy with the training dataset, consider the bounds of each variable!
    * Select the optimal strategy and describe it thoroughly (X, y variables used, a brief description of the ML models, results).
    * Now, use the optimal strategy with the test dataset and compare it to a passive strategy.

-------

In [1]:
from technical_analysis import Operation, TradingStrategy, MLModels

In [2]:
strategy = TradingStrategy('5m')

In [3]:
strategy.prepare_data_for_ml()

In [4]:
strategy.train_df

,Pt-1,Pt-2,Pt-3,Volatility,Returns,Spread,Buy_Signal_xgb,Sell_Signal_xgb
0,130.889999,130.509902,130.875000,0.003053,0.000350,0.270004,0,1
1,130.935806,130.889999,130.509902,0.002420,0.000900,0.360000,0,1
2,131.053604,130.935806,130.889999,0.002407,-0.002820,0.639999,1,0
3,130.684005,131.053604,130.935806,0.002677,0.003299,0.580002,0,1
4,131.115097,130.684005,131.053604,0.002249,-0.000535,0.279999,0,1
...,...,...,...,...,...,...,...,...
31307,164.869995,165.060104,165.029998,0.000849,0.000637,0.260010,0,1
31308,164.975006,164.869995,165.060104,0.000881,-0.000576,0.230011,0,1
31309,164.880004,164.975006,164.869995,0.001483,-0.003700,1.380004,0,1
31310,164.270004,164.880004,164.975006,0.001488,-0.000670,0.479004,1,0


In [5]:
strategy.test_df

,Pt-1,Pt-2,Pt-3,Volatility,Returns,Spread,Buy_Signal_xgb,Sell_Signal_xgb
31312,164.380004,164.160003,164.270004,0.001426,0.000608,0.639893,0,1
31313,164.479995,164.380004,164.160003,0.001519,-0.002007,0.669998,0,1
31314,164.149902,164.479995,164.380004,0.001489,-0.001461,0.419998,1,0
31315,163.910003,164.149902,164.479995,0.001633,0.001556,0.470398,1,0
31316,164.164993,163.910003,164.149902,0.001711,-0.002223,0.429901,1,0
...,...,...,...,...,...,...,...,...
39135,128.244995,128.218902,128.316894,0.000853,-0.000624,0.220002,1,0
39136,128.164993,128.244995,128.218902,0.000664,0.000975,0.240005,1,0
39137,128.289993,128.164993,128.244995,0.000656,-0.000078,0.190003,1,0
39138,128.279998,128.289993,128.164993,0.001072,0.002806,0.485093,1,0


In [6]:
strategy.prepare_data_for_log_model()

In [7]:
strategy.vtrain_data

,Returns,Volatility,Close_Trend,Volume_Trend,Spread,LR_Buy_Signal,LR_Sell_Signal
0,0.000350,0.003053,-0.169407,-80017.878788,0.270004,0,1
1,0.000900,0.002420,-0.150417,-84001.569697,0.360000,0,1
2,-0.002820,0.002407,-0.130117,-61227.557576,0.639999,0,1
3,0.003299,0.002677,-0.056956,-86349.872727,0.580002,0,1
4,-0.000535,0.002249,-0.017863,-102522.878788,0.279999,0,1
...,...,...,...,...,...,...,...
29350,-0.000678,0.001662,-0.002851,13608.278788,0.289994,0,1
29351,0.000688,0.001567,-0.006465,5544.478788,0.245011,0,1
29352,-0.000072,0.001554,-0.006207,-1463.545455,0.229996,0,0
29353,0.001917,0.001665,0.008027,-2813.921212,0.361191,0,0


In [8]:
strategy.optimize_and_fit_models()

[I 2024-03-10 15:49:04,354] A new study created in memory with name: no-name-6e3e002c-b422-47e1-9d1e-fca88f2f72f5
[I 2024-03-10 15:49:22,268] Trial 0 finished with value: 0.291354663036079 and parameters: {'C': 284.78879984757157, 'l1_ratio': 0.46203779371656184, 'fit_intercept': True}. Best is trial 0 with value: 0.291354663036079.
[I 2024-03-10 15:49:40,229] Trial 1 finished with value: 0.291354663036079 and parameters: {'C': 1027.7001477268836, 'l1_ratio': 0.6826538747957638, 'fit_intercept': True}. Best is trial 0 with value: 0.291354663036079.
[I 2024-03-10 15:49:58,216] Trial 2 finished with value: 0.291354663036079 and parameters: {'C': 310.1321388174293, 'l1_ratio': 0.286654032318962, 'fit_intercept': True}. Best is trial 0 with value: 0.291354663036079.
[I 2024-03-10 15:50:16,198] A new study created in memory with name: no-name-1f10bb8a-e36f-41ec-a99d-38df8ea726ef
[I 2024-03-10 15:50:32,544] Trial 0 finished with value: 0.2985678180286437 and parameters: {'C': 0.0011835755127

In [9]:
strategy.data.columns

Index(['Unnamed: 0', 'Timestamp', 'Gmtoffset', 'Datetime', 'Open', 'High',
       'Low', 'Close', 'Volume', 'Returns', 'Volatility', 'Close_Trend',
       'Volume_Trend', 'Spread', 'Future_Return_Avg_5', 'LR_Buy_Signal',
       'LR_Sell_Signal', 'Pt-1', 'Pt-2', 'Pt-3', 'Future_Price',
       'Buy_Signal_xgb', 'Sell_Signal_xgb', 'Logistic_Buy_Signal',
       'Logistic_Sell_Signal', 'XGBoost_buy_signal', 'XGBoost_sell_signal'],
      dtype='object')

---

## <font color='navy'>  Order 1 <font color='black'> 

In this section, we'll demonstrate a practical example using a 5-minute time frame. Here's how we'll proceed:
    
* Initialize by declaring the chosen time frame.
* Test all combinations.
* Plot the result of the best one.
* Optimize the parameters of the best indicators, as well as the take_profit, stop_loss, and number of shares.
* Plot the results of the best optimized combination.
* Plot the best optimized combination on test data with the same time frame.

### <font color='navy'>  Run all combinations <font color='black'> 

In [10]:
from technical_analysis import Operation, TradingStrategy
strategy = TradingStrategy('5m')
strategy.run_combinations()

Ejecutando con combinación de indicadores: ['Logistic']


KeyError: "None of [Index(['Logistic_buy_signal'], dtype='object')] are in the [columns]"

For this timeframe, the best combination is: 'SAR' and 'ADX'. 

### <font color='navy'>  Backtesting of the best combination <font color='black'> 

Here we test a trading strategy on historical data to determine its potential for future success. It allows traders to simulate a strategy's performance without risking actual capital, providing insights into the strategy's risk and profitability before live implementation.

In [ ]:
strategy.plot_results(best = True)

### <font color='navy'>  Best model without optimization on test <font color='black'> 

In [ ]:
strategy.test()

In [ ]:
strategy.strategy_value[-1]

### <font color='navy'>  Parameters Optimization <font color='black'> 

In [ ]:
strategy.optimize_parameters()

### <font color='navy'>  Strategy backtest with optimized paramters <font color='black'> 

In [ ]:
strategy.plot_results(best = True)

### <font color='navy'>  Best optimized strategy | Final value <font color='black'> 

In [ ]:
strategy.strategy_value[-1]

### <font color='navy'>  Run and backtest best strategy | Test data <font color='black'> 

In [ ]:
strategy.test()

### <font color='navy'>  Test final value <font color='black'>

In [ ]:
strategy.strategy_value[-1]

In the first chart, "Best model without optimization on test," we see the strategy's value over a series of trades without any parameter tuning. The performance appears relatively stable, hovering around the initial strategy value, with no significant growth over the number of trades executed. This suggests that while the strategy may be consistent, it lacks the efficacy to generate substantial returns as is.

Contrastingly, the second chart, "Strategy backtest with optimized parameters," displays the strategy's performance after optimization. There is a noticeable difference in the strategy value, which shows greater variability and peaks that exceed the initial value by a more significant margin. This implies that the optimization process has potentially enhanced the strategy's ability to capitalize on profitable opportunities in the market.

The comparison between the two suggests that optimization can have a marked impact on a trading strategy's success, enhancing its ability to generate returns. However, the increased variability also hints at potentially higher risk, which underscores the importance of risk management in trading strategy development. The results also serve as a reminder that past performance is not indicative of future results, and even an optimized strategy should be approached with caution and monitored closely.

## Pasive Strategy
Passive investing is an investment strategy to maximize returns by minimizing buying and selling.

In [ ]:
# Convert the 'Date' column to datetime type
data['Date'] = pd.to_datetime(data['Date'])

# Get the closing price of the first and last data
first_close = data.iloc[0]['Close']
last_close = data.iloc[-1]['Close']

# Calculate the passive asset return
passive_return = (last_close - first_close) / first_close

print("The passive asset return from the first close to the last close is: {:.2%}".format(passive_return))

# Comparison with the strategy used
cash = 1000000
cash_final = 1107153.05
strategy_return = (cash_final - cash) / cash
print("The strategy return from the first close to the last close is: {:.2%}".format(strategy_return))

# Sort the data by date if they are not sorted
data = data.sort_values(by='Date')
plt.figure(figsize=(12, 8))
plt.plot(data['Date'], data['Close'], label='Closing Price', color='blue')
plt.title('Closing Price of the Asset')
plt.xlabel('Date')
plt.ylabel('Closing Price')
plt.legend()
plt.grid(True)
plt.show()

# Calculate the cumulative return
data['Returns'] = data['Close'].pct_change().fillna(0)

# Calculate the cumulative value
initial_investment = cash
data['Investment_Value'] = (1 + data['Returns']).cumprod() * initial_investment
plt.figure(figsize=(12, 8))
plt.plot(data['Date'], data['Investment_Value'], label='Investment Value', color='green')
plt.title('Investment Return')
plt.xlabel('Date')
plt.ylabel('Investment Value')
plt.legend()
plt.grid(True)
plt.show()

final_value = data['Investment_Value'].iloc[-1]
print("The final value of the passive investment: ${:,.2f}".format(final_value))


The passive asset, represented by the underlying security, demonstrated a substantial return of 18.54% from the initial to the final closing prices, indicating favorable market conditions over the observed period. On the other hand, the implemented strategy yielded a lower return of 10.72%, suggesting that while it still produced positive results, it underperformed compared to simply holding the asset passively. This observation prompts a critical examination of the strategy's effectiveness and highlights the importance of continuously evaluating and adjusting investment strategies to optimize returns in evolving market environments.